# Understand paper from scratch

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import pickle
from os.path import dirname
import sys

In [10]:
## add python modules
#import sys
sys.path.append('libs/')
from parameters import *

sys.path.append(DATA_PATH)
sys.path.append('CDN_Molecule/')

#from preprocess import *
import preProcess
import model

# Preprocessing

In [43]:
preProcess.process_data()

NameError: name 'input_file' is not defined

In [11]:
FLAGS = tf.app.flags.FLAGS

## Loading data

The dataset are 250k randomly chosen drug-like molecules from the ZINC database.

In [12]:
# Load data, by default from data/TrainVectors.pickle
print(f"Loading data from file '{FLAGS.data_file}' ...")
x = np.array(pickle.load(open(FLAGS.data_file, "rb")))
print("done")

Loading data from file '../CDN_Molecule/data/TrainVectors_python3.pickle' ...
done


In [13]:
x.shape
print(f" We have {x.shape[0]} molecules, with {x.shape[1]} features each")

 We have 173196 molecules, with 50 features each


## Interpretation of input

In [14]:
x[:2]

array([[34,  0,  1,  0,  2,  2,  3,  4,  5,  0,  0,  6,  7,  8,  9,  8,
        10,  8,  8,  6, 11,  8, 12, 10, 10,  8, 10, 12,  0, 13,  8,  9,
        13,  0,  1,  0,  2,  2,  3,  4,  6,  0, 13,  0,  5, 35, 36, 36,
        36, 36],
       [34,  0,  0,  1,  7,  3, 19,  4,  6,  0,  0, 13,  1,  0,  2,  4,
         6,  0, 13,  6,  0,  0, 13,  1,  0,  2,  3,  4,  6, 15, 13,  8,
         5,  8, 20,  8,  8,  5, 21, 22, 35, 36, 36, 36, 36, 36, 36, 36,
        36, 36]], dtype=int32)

## Shuffle data

In [15]:
# Shuffle data -- ok, shuffle indices of rows and create a copy, it's memory effective
np.random.seed(10)
print(f"np.arange(len(x)) = {np.arange(len(x))}\nlen(x) = {len(x)}")
shuffle_indices = np.random.permutation(np.arange(len(x)))
x_shuffled = x[shuffle_indices]

np.arange(len(x)) = [     0      1      2 ... 173193 173194 173195]
len(x) = 173196


In [16]:
print(f"-1 * int({FLAGS.dev_sample_percentage}) * float({x_shuffled.shape[0]})")
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(x_shuffled.shape[0]))
print(f"x_shuffled[:{dev_sample_index}], x_shuffled[{dev_sample_index}:]")
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
print(("Train/Dev split: {:d}/{:d}".format(len(x_train), len(x_dev))))

-1 * int(0.03) * float(173196)
x_shuffled[:-5195], x_shuffled[-5195:]
Train/Dev split: 168001/5195


### Split input

In [17]:
def split_input(batch):
    """
    This looks like "shifting" each row (instance) by one position to the left and adding 36 at the end,
    for example
    [34  0 16  0  6  0 13  0 10  5  8  6 16 15 13  8  9  8  6 10  8 12 10  9
      0  0  0  7 12  8  9  8  8  8  8  8  9 15  0 13 10  6  0 13  8  5 16 15
     35 36]

    becomes

    [ 0 16  0  6  0 13  0 10  5  8  6 16 15 13  8  9  8  6 10  8 12 10  9  0
      0  0  7 12  8  9  8  8  8  8  8  9 15  0 13 10  6  0 13  8  5 16 15 35
     36 36]

    :param current_batch_as_ndarray: 64 rows -- 64 instances per batch, each instance has 50 dimensions
    :return: tuple: the parameter itself (current_batch_as_ndarray) and the shifted one, see above
    """

    # There are 64 rows -- 64 instances per batch, each instance has 50 dimensions
    # print("split_input(), current_batch_as_ndarray.shape:", current_batch_as_ndarray.shape)

    # simply copy current_batch_as_ndarray to x_batch output
    x_batch = batch

    # remove the first position and add 36 to the end
    y_batch = np.concatenate([x_batch[:, 1:], np.zeros(shape=[x_batch.shape[0], 1], dtype=np.int32) + 36],
                             axis=1)

    return x_batch, y_batch

In [34]:
x_batch, y_batch = split_input(x_train[:1])

In [35]:
x_batch

array([[34, 15, 16,  0,  6,  1, 15, 11,  4, 13,  8,  5,  8,  8,  8,  6,
        11,  8,  9,  8,  8,  8,  6,  0, 18, 13,  8,  8,  9, 13,  8,  8,
         5, 24, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36]], dtype=int32)

In [36]:
y_batch

array([[15, 16,  0,  6,  1, 15, 11,  4, 13,  8,  5,  8,  8,  8,  6, 11,
         8,  9,  8,  8,  8,  6,  0, 18, 13,  8,  8,  9, 13,  8,  8,  5,
        24, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36]], dtype=int32)

# The model

In [19]:
import tensorflow as tf


class CDN:

    def __init__(self, sequence_length, vocab_size, embedding_size, filter_sizes, num_filters, max_molecule_length,
                 gaussian_samples, variational=True, l2_reg_lambda=0.5, generation_mode=False, test_mode=False):
# define some variables
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.max_molecule_length = max_molecule_length
        self.l2_reg_lambda = l2_reg_lambda
        self.gaussian_samples_dim = gaussian_samples
        self.variational = variational
        self.encoder_input_GO = tf.placeholder(tf.int32, [None, sequence_length], name="encoder_input")
        self.encoder_input = tf.placeholder(tf.int32, [None, sequence_length], name="encoder_input")  ## no go
        self.gaussian_samples = tf.placeholder(tf.float32, [None, self.gaussian_samples_dim], name="unit_gaussians")
        self.generation_mode = generation_mode
        self.test_mode = test_mode
# how to encode 
    def encode(self):
        # Embedding layer
        with tf.name_scope("embedding"):
            self.E = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_size], -1.0, 1.0), name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.E, self.encoder_input)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            self.embedded_chars_go = tf.nn.embedding_lookup(self.E, self.encoder_input_GO)

        # Create a convolution layers for each filter size
        conv_flatten = []
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[self.num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                conv_flatten.append(tf.contrib.layers.flatten(h))
        conv_output = tf.concat(conv_flatten, axis=1)

        # Flatten feature vector
        h_pool_flat3 = tf.nn.relu(tf.contrib.layers.linear(conv_output, 450))

        if self.variational:
            with tf.name_scope("Variational"):
                self.z_mean = tf.contrib.layers.linear(h_pool_flat3, 300)
                self.z_stddev = tf.contrib.layers.linear(h_pool_flat3, 300)
                latent_loss = 0.5 * tf.reduce_sum(
                    tf.square(self.z_mean) + tf.square(self.z_stddev) -
                    tf.log(tf.square(self.z_stddev) + 1e-10 ) - 1,
                    1
                )
                self.mean_latent_loss = tf.reduce_mean(latent_loss)
                if self.generation_mode:
                    h_pool_flat = self.gaussian_samples
                else:
                    h_pool_flat = self.z_mean + (self.z_stddev * self.gaussian_samples)

                h_pool_flat = tf.identity(h_pool_flat, "encoded_final")

        return h_pool_flat, self.mean_latent_loss

    def decode_rnn(self, z):
        def pick_next_argmax(former_output, step):
            next_symbol = tf.expand_dims(tf.stop_gradient(tf.argmax(former_output, 1)), axis=-1)
            return tf.nn.embedding_lookup(self.E, next_symbol), next_symbol

        def pick_next_top_k(former_output, step):
            next_symbol = tf.multinomial(former_output, 1)
            return tf.nn.embedding_lookup(self.E, next_symbol), next_symbol

        with tf.name_scope("Decoder"):
            self.decode_start = tf.nn.relu(tf.contrib.layers.linear(z, 150))
            decoder_inputs_list = tf.split(self.embedded_chars_go, self.max_molecule_length, axis=1)
            decoder_inputs_list = [tf.squeeze(i, axis=1) for i in decoder_inputs_list]
            rnn_cell = tf.nn.rnn_cell.LSTMCell(150, state_is_tuple=False)

            self.lstm_outputs = []
            temp_logits = []
            self.all_symbols = []
            symbol = tf.ones(1)  # output for test mode
            for i in range(self.max_molecule_length):
                if not self.test_mode or i == 0:
                    if i == 0:
                        output, state = rnn_cell(decoder_inputs_list[i], state=z)
                    else:
                        output, state = rnn_cell(decoder_inputs_list[i], state=state)
                else:
                    next_decoder_input, symbol = pick_next_argmax(temp_logits[-1], i)
                    next_decoder_input = tf.squeeze(next_decoder_input, axis=1)
                    output, state = rnn_cell(next_decoder_input, state=state)
                with tf.variable_scope("decoder_output_to_logits") as scope_logits:
                    if i > 0:
                        scope_logits.reuse_variables()
                    temp_logits.append(tf.contrib.layers.linear(output, self.vocab_size))

                self.lstm_outputs.append(output)
                if i > 0:
                    self.all_symbols.append(symbol)
                if i == self.max_molecule_length - 1 and self.test_mode:
                    self.all_symbols.append(pick_next_argmax(temp_logits[-1], i)[1])
            if self.test_mode:
                self.all_symbols = tf.squeeze(tf.transpose(tf.stack(self.all_symbols), [1,0,2]), axis=-1)

            self.decoder_logits = tf.transpose(tf.stack(temp_logits), perm=[1, 0, 2])
            self.decoder_prediction = tf.argmax(self.decoder_logits, 2, name="decoder_predictions")

            return self.decoder_logits

    def loss(self, logits, latent_loss):
        with tf.name_scope("loss"):
            self.output_onehot = tf.one_hot(self.encoder_input, self.vocab_size)
            self.losses = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.output_onehot)
            self.CE_loss = tf.reduce_mean(self.losses)
            self.total_loss = self.CE_loss + .00001 * latent_loss

        with tf.name_scope("accuracy"):
            decoder_prediction = tf.argmax(logits, 2, name="decoder_predictions")
            x_target = tf.to_int64(self.encoder_input)
            correct_predictions = tf.equal(decoder_prediction, x_target)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

        return self.total_loss, self.accuracy

In [20]:
cnn = model.CDN(
            sequence_length=x_train.shape[1],
            vocab_size=FLAGS.vocab_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            max_molecule_length=FLAGS.max_molecule_length,
            gaussian_samples=FLAGS.unit_gaussian_dim,
            l2_reg_lambda=FLAGS.l2_reg_lambda,
            variational=True,
            test_mode=False,
            generation_mode=False
        )

In [24]:
from tensorflow.python.layers import base
import tensorflow as tf
import tensorflow.contrib.slim as slim
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

AttributeError: 'CDN' object has no attribute 'trainable_variables'

In [32]:
tf.summary


<module 'tensorflow.summary' from '/home/schreibf/anaconda3/envs/cdn_molecule/lib/python3.6/site-packages/tensorflow/summary/__init__.py'>

# Training

What the code does

1. instantiate model
2. set learning rate
3. set adam optimizer
4. Instantiate Encoder
5. Instantiate Decoder
6. Instantiate Loss function (optimize gradients)
7. Define summaries
8. Generate batches
9. Run training over all batches

In [33]:
# in the scope of a default TF graph... why? TL;DR: It's unnecessary, but it's a good practice to follow.
# See https://stackoverflow.com/a/39616491
with tf.Graph().as_default():
    sess_config = tf.ConfigProto(
        gpu_options=tf.GPUOptions(allow_growth=True)
    )
    sess = tf.Session(config=sess_config)
    # here we begin the tensorflow session
    with sess.as_default():
        cnn = model.CDN(
            sequence_length=x_train.shape[1],
            vocab_size=FLAGS.vocab_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            max_molecule_length=FLAGS.max_molecule_length,
            gaussian_samples=FLAGS.unit_gaussian_dim,
            l2_reg_lambda=FLAGS.l2_reg_lambda,
            variational=True,
            test_mode=False,
            generation_mode=False
        )
        # cnn is a simple CDN object, nothing inherited from TF classes
        assert isinstance(cnn, model.CDN)

        # TODO why do we need a global_step non-trainable variable?
        # According to this StackOverflow, it's a way to train AdamOptimizer with custom learning rate, see
        # https://stats.stackexchange.com/questions/200063/adam-optimizer-with-exponential-decay
        global_step = tf.Variable(1, name="global_step", trainable=False)

        # A scalar `Tensor` of the same type as `learning_rate`.  The decayed learning rate.
        learning_rate = tf.train.exponential_decay(learning_rate=FLAGS.initial_learning_rate, global_step=global_step,
                                                   decay_steps=5000, decay_rate=0.96, staircase=True)

        # somehow confusing how the learning rate is passed to the AdamOptimizer...
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # but we know the type: it's AdamOptimizer instance, inherited from Optimizer
        assert isinstance(optimizer, tf.train.AdamOptimizer)

        # CDN.encode() defines the computational graph - returns two tensors
        # TODO explore encode() in detail
        
        #####
        # Encoding part
        #####
        # returns pooling and loss function (mean_latent_loss)
        encoded, latent_loss = cnn.encode()
        assert isinstance(encoded, tf.Tensor)
        assert isinstance(latent_loss, tf.Tensor)
        
        #####
        # Decoder part
        #####
        # another Tensor output from the decoder
        logits = cnn.decode_rnn(encoded)
        assert isinstance(logits, tf.Tensor)

        #####
        # loss function
        #####
        # and CDN.loss() returns two Tensors: loss and accuracy
        loss, accuracy = cnn.loss(logits, latent_loss)
        assert isinstance(loss, tf.Tensor)
        assert isinstance(accuracy, tf.Tensor)

        # grads_and_vars is a list
        # It returns a list of (gradient, variable) pairs where "gradient" is the gradient for "variable".
        # It is the first part of minimize()
        grads_and_vars = optimizer.compute_gradients(loss)

        # The second part of minimize() which applies gradients to variables
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        assert isinstance(train_op, tf.Operation)

        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory
        timestamp = str(int(time.time()))
        if FLAGS.parameters_file is not None:
            out_dir = os.path.abspath(os.path.join(os.path.curdir, FLAGS.parameters_file))
        else:
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print(("Writing to {}\n".format(out_dir)))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", loss)
        acc_summary = tf.summary.scalar("accuracy", accuracy)
        kl_div_summary = tf.summary.scalar("KL-div", latent_loss)
        CE_loss_summary = tf.summary.scalar("CE-loss", cnn.CE_loss)
        learning_rate_summary = tf.summary.scalar("learningRate", learning_rate)

        # Train Summaries
        train_summary_op = tf.summary.merge(
            [loss_summary, acc_summary, grad_summaries_merged, kl_div_summary, CE_loss_summary,
             learning_rate_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train") # file directory
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph) # save in file

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary, kl_div_summary, CE_loss_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory.
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)
        
        # Resuming training
        if (FLAGS.load_param) and (FLAGS.parameters_file is not None):
            # model file to restart training
            checkpoint_file = FLAGS.parameters_file
            saver.restore(sess, checkpoint_file)
            print("Loaded: " + str(checkpoint_file))
        else:
            sess.run(tf.global_variables_initializer())

        # Generate batches - a batch iterator for a data set
        num_epochs  = 1
        #batches = preProcess.batch_iter(x_train, FLAGS.batch_size, FLAGS.num_epochs)
        batches = preProcess.batch_iter(x_train, FLAGS.batch_size, num_epochs)
        # what is actually 'batches'? - it is a generator
        # a batch is defined as an x and a y.
        
        for idx, batch in enumerate(batches):
            current_step = tf.train.global_step(sess, global_step)
            # split each batch into x and y
            x_bat, y_bat = split_input(batch)
            # feed data to placeholders, so data that changes with each batch
            
            feed_dict = {
                # these are CDN placeholders
                cnn.encoder_input: y_bat,
                cnn.encoder_input_GO: x_bat,
                cnn.gaussian_samples: np.random.normal(size=[x_bat.shape[0], FLAGS.unit_gaussian_dim]),
            }

            # Runs operations and evaluates tensors in `fetches`.
            # TODO Why the heck do we need to call train_op AND total_loss, summary, etc.? Just for debugging?
            _, step, summaries, loss, accuracy, kldiv, CEloss = sess.run(
                fetches=[train_op, global_step, train_summary_op, cnn.total_loss, cnn.accuracy, cnn.mean_latent_loss,
                         cnn.CE_loss], feed_dict=feed_dict)

            time_str = datetime.datetime.now().isoformat()
            if current_step % 100 == 0:
                print(("{}: step {}, loss {:g}, latentLoss: {:g}, reconstructionLoss: {:g}, acc {:g}".format(
                    time_str, step, loss, kldiv, CEloss, accuracy)))

            train_summary_writer.add_summary(summaries, step)

            # every N steps we run also evaluation
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")

                x_bat, y_bat = split_input(batch)
                feed_dict = {
                    cnn.encoder_input: y_bat,
                    cnn.encoder_input_GO: x_bat,
                    cnn.gaussian_samples: np.random.normal(size=[x_bat.shape[0], FLAGS.unit_gaussian_dim]),
                }

                step, summaries, loss, accuracy, kldiv, CEloss = sess.run(
                    [global_step, dev_summary_op, cnn.total_loss, cnn.accuracy, cnn.mean_latent_loss, cnn.CE_loss],
                    feed_dict)

                time_str = datetime.datetime.now().isoformat()
                print(
                    ("{}: step {}, loss {:g}, klDiv: {:g}, CE-loss: {:g}, acc {:g}".format(time_str, step, loss, kldiv,
                                                                                           CEloss, accuracy)))
                if dev_summary_writer:
                    dev_summary_writer.add_summary(summaries, step)

                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print(("Saved model checkpoint to {}\n".format(path)))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:t